## Lambda function for airport and demographical data 

In [ ]:
import json
import pytz
import requests
import pandas as pd
from datetime import datetime

def lambda_handler(event, context):
    # Database connection details
    schema = "Gans_support_data"
    host = "your_host_address"
    user = "admin"
    password = "PASSWORD"  # os.environ.get('PASSWORD')
    port = 3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    # List of example cities to collect weather data
    cities = ['Berlin', 'Hannover', 'Bremen', 'Hamburg', 'Munich', 'Dresden', 'Paris', 'Barcelona', 'Lisbon', 'Brussels', 'Amsterdam', 'Budapest']
    cities_df = pd.DataFrame()

    # Loop through cities and collect weather data, then append it to cities_df
    for city in cities:
        city_data = get_cities(city)  # get_cities() returns a pandas DataFrame
        cities_df = pd.concat([cities_df, city_data])

    # Save cities data to the database
    cities_df.to_sql('cities', if_exists='append', con=con, index=False)

    # Retrieve airports data using latitude and longitude from cities_df
    lats = cities_df['lat']
    lons = cities_df['lon']
    airports_df = retrieve_airport(lats, lons)

    # Save airports data to the database
    airports_df.to_sql('airports', if_exists='append', con=con, index=False)

    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

def get_cities(city):
    # Define timezone for the city data
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)

    # API key for OpenWeatherMap
    API_key = "Your_API_Key"

    # Collect weather information for the specified city
    weathers = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric").json()

    # Normalize the 'city' data into cities_df DataFrame
    cities_df = pd.json_normalize(weathers['city'])

    # Rename columns in the cities_df
    cities_df.rename(columns={'name': 'city', 'coord.lat': 'lat', 'coord.lon': 'lon'}, inplace=True)

    # Add 'timestamp_population' column to the cities_df
    cities_df['timestamp_population'] = now.strftime("%Y")

    # Remove unwanted columns from cities_df
    columns_to_delete = ['timezone', 'sunrise', 'sunset', 'id']
    cities_df.drop(columns=columns_to_delete, inplace=True)

    return cities_df

def retrieve_airport(lat, lon):
    list_for_df = []

    # Loop through latitudes and longitudes to retrieve airports data
    for lat_value, lon_value in zip(lat, lon):
        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        querystring = {"lat": lat_value, "lon": lon_value, "radiusKm": "50", "limit": "10", "withFlightInfoOnly": "true"}
        headers = {
            "X-RapidAPI-Key": "Your_API_Key",
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        # Make the API request to retrieve airports data
        response = requests.get(url, headers=headers, params=querystring)
        list_for_df.append(pd.json_normalize(response.json()['items']))

    # Concatenate the list of DataFrames into airports_df
    airports_df = pd.concat(list_for_df, ignore_index=True)

    # Rename columns in airports_df
    airports_df.rename(columns={'name': 'airport', 'municipalityName': 'municipality'}, inplace=True)

    # Remove unwanted columns from airports_df
    columns_to_delete = ['iata', 'shortName', 'countryCode', 'location.lat', 'location.lon']
    airports_df.drop(columns=columns_to_delete, inplace=True)

    return airports_df


## Lambda function for collecting Flight data 

In [ ]:
import json
import requests
import pandas as pd
from datetime import datetime, date, timedelta
from pytz import timezone

def lambda_handler(event, context):
    # Database connection details
    schema = "Gans_support_data"
    host = ""
    user = "admin"
    password = "PASSWORD"  # os.environ.get('PASSWORD')
    port = 3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    # TODO implement any other required functionality

    # Initialize an empty DataFrame for flights data
    flights_df = pd.DataFrame()

    # Retrieve flights data for the given airports and store it in a DataFrame
    flights_df = retrieve_flights(airports_df['icao'])

    # Save the flights data to the database
    flights_df.to_sql('flights', if_exists='append', con=con, index=False)

    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

def retrieve_flights(icao_list):
    # Get the current date in the 'Europe/Berlin' timezone
    today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
    # Calculate tomorrow's date
    tomorrow = (today + timedelta(days=1))

    # Create an empty list to store flights data
    df_list = []

    # Loop over each airport code in the provided list
    for icao in icao_list:
        # Define time periods for flight retrieval
        trigger_times = [["00:00", "11:59"], ["12:00", "23:59"]]

        # Loop over each time period
        for time in trigger_times:
            # Construct the URL for the API request
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
          

            # Set query parameters for the API request
            querystring = {
                "withLeg": "true",
                "direction": "Arrival",
                "withCancelled": "false",
                "withCodeshared": "true",
                "withCargo": "false",
                "withPrivate": "false"
            }
            # Set headers for the API request
            headers = {
                'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
                'x-rapidapi-key': "Your_API_Key"
            }
            # Make the API request to retrieve flights data
            response = requests.get(url, headers=headers, params=querystring)

            # Convert the response to a JSON object
            flights_json = response.json()

            # Loop over each flight in the arrivals data and extract relevant information
            for flight in flights_json['arrivals']:
                # Create a dictionary to store flight information
                flights_dict = {}
                # Populate the dictionary with flight data
                flights_dict['arrival_icao'] = icao
                flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
                flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
                flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
                flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
                flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
                flights_dict['airline'] = flight['airline'].get('name', None)
                flights_dict['flight_number'] = flight.get('number', None)
                flights_dict['data_retrieved_at'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()

                # Append the flight information to the list
                df_list.append(flights_dict)

    # Convert the list of flight dictionaries to a DataFrame and return it
    return pd.DataFrame(df_list)


## Lambda function for collecting Weather data 

In [ ]:
import json
import pytz
import requests
import pandas as pd
from datetime import datetime


def lambda_handler(event, context):
    schema="Gans_support_data"
    host=""
    user="admin"
    password="PASSWORD"#os.environ.get('PASSWORD')
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    # TODO implement

    cities = ['Berlin', 'Hannover', 'Bremen', 'Hamburg', 'Munich','Dresden', 'Paris', 'Barcelona', 'Lisbon', 'Brussels', 'Amsterdam', 'Budapest']

    weathers_df = pd.DataFrame()

    for city in cities:
        weather_data = get_weather(city)  # get_weather() returns a pandas DataFrame
        weathers_df = pd.concat([weathers_df, weather_data])

    weathers_df.to_sql('weather', if_exists='append', con=con, index=False)


    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

def get_weather(city):
   try:
    # Define timezone
      tz = pytz.timezone('Europe/Berlin')
      now = datetime.now().astimezone(tz)

    # Define the parameters to be passed into the url.
      API_key = "Your_API_Key"

    # Collecting weather information
      weathers = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric").json()


    # Create an empty DataFrame for weather data
      weather_df = pd.DataFrame()

    # Extract weather data and concatenate into weather_df
      for i in range(weathers['cnt']):
        new_row = pd.DataFrame(weathers['list'][i]['weather'])
        weather_df = pd.concat([weather_df, new_row], ignore_index=True)

    # Normalize the 'list' data into lists_norm DataFrame
      lists_norm = pd.json_normalize(weathers['list'])

    # Create the weathers_df DataFrame by dropping the 'weather' column from lists_norm
    # The 'weather' column contains nested data, and it was extracted separately in the previous loop
      weathers_df = lists_norm.drop(['weather'], axis=1)

    # Add 'information_retrieved_at' column to the weathers_df
      weathers_df['information_retrieved_at'] = now.strftime("%d/%m/%Y %H:%M:%S")

    # Joining two dataframes
      weathers_df = weathers_df.join(weather_df)

    # Remove the unwanted columns from weathers_df
      columns_to_delete = ['dt', 'visibility', 'pop', 'main.sea_level', 'wind.gust', 'main.grnd_level',
                         'main.temp_kf', 'sys.pod', 'id', 'main', 'description', 'icon']
      for column_name in columns_to_delete:
        del weathers_df[column_name]

    # Rename columns' names in the weathers_df
      weathers_df.rename(columns={'dt_txt':'forecast_time', 'main.temp':'temperature',
                                'main.feels_like':'temp_feels_like', 'main.temp_min':'temp_min',
                                'main.temp_max':'temp_max', 'main.pressure':'pressure',
                                'main.humidity':'humidity', 'clouds.all':'clouds',
                                'wind.speed':'wind_speed', 'wind.deg':'wind_deg', 'rain.3h':'rain_3h'}, inplace=True)


      return weathers_df

   except Exception as e:
        print(f"An error occurred while retrieving weather data for {city}: {e}")
        return None